In [25]:
#install the Pomegranate package by typing 'pip install pomegranate' or following other instructions from here:
#     https://pomegranate.readthedocs.io/en/latest/install.html
from pomegranate import *
import numpy as np

#copy the file containing the DNA sequence into the folder of this notebook (or add a path below)
#read the DNA sequence into a string
file = open('../original_files/exercises/Sheet11/BlochmannDNA.txt', 'r')
dna = file.readline()
print('Length of DNA sequence:',len(dna))
file.close()
# Fun fact: Python can handle enormous strings. You could do the same with much larger genomes. See
# https://stackoverflow.com/questions/1739913/what-is-the-max-length-of-a-python-string

Length of DNA sequence: 791654


Your task: 
Set up a Hidden Markov Model describing the distribution of a,c,g,t nucleotids in exon and intron states.
The nucleotids are the observable states, being exon or intron are the hidden states. Do this as follows:

### A) 
You know that positions 405000 to 405071 and 690040 to 690159 are exons. Compute the percentages of a,c,g,t nucleotids in those positions. The results can later be taken as emission probabilities for the exon state.

Likewise you know that positions 1040 to 1111, 2060 to 2151, and 386370 to 386449 are introns. Compute the percentages of a,c,g,t nucleotids in those positions. The results can later be taken as emission probabilities for the intron state.

The outcomes should be probability distributions, so make sure that everything sums up to 1 (just round somewhere if necessary).

Print out your results.

In [26]:
exons = dna[405000:405071]+dna[690040:690159]
introns = dna[1040:1111]+dna[2060:2151]+dna[386370:386449]

dna_strings = {'exons':exons, 'introns':introns}
b = {'exons':{}, 'introns':{}}
for i in ['exons', 'introns']:
    for j in ['a', 'c', 'g', 't']:
        b[i][j] = (round(dna_strings[i].count(j)/len(dna_strings[i]), 4))

b

{'exons': {'a': 0.2842, 'c': 0.2211, 'g': 0.2579, 't': 0.2368},
 'introns': {'a': 0.2365, 'c': 0.083, 'g': 0.1494, 't': 0.5311}}

### B) 
Define a Pomegranate Hidden Markov Model. The probability of passing from an intron to an exon state or vice versa is 0.1. As starting probabilities take 0.5 for both states (not true in biology, but we do this because in part C) we want to analyze pieces from the middle of our whole DNA sequence, not just initial segments...)

In [27]:
model = HiddenMarkovModel(name = "DNA model")

# emission matrix
exons_em = DiscreteDistribution(b['exons'])
exons_st = State(exons_em, name = 'exons')
introns_em = DiscreteDistribution(b['introns'])
introns_st = State(introns_em, name = 'introns')
# add states to the model
model.add_states(exons_st, introns_st)


# initial distribution
model.add_transition(model.start, exons_st, 1/2)
model.add_transition(model.start, introns_st, 1/2)

# transition
model.add_transition(exons_st, exons_st, 0.9)
model.add_transition(introns_st, introns_st, 0.9)
model.add_transition(exons_st, introns_st, 0.1)
model.add_transition(introns_st, exons_st, 0.1)

# finalize
model.bake()


In [28]:
model.dense_transition_matrix()

array([[0.9, 0.1, 0. , 0. ],
       [0.1, 0.9, 0. , 0. ],
       [0.5, 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. ]])

### C) 
Print out the most likely underlying hidden state sequences for the following observed sequences:

In [34]:
seq1='aaaagcgcgttgagactagcattttattatttttgggtt'
seq2='catttatttttatatactaaaggcgggcgcttaatgtgcgatgaaataatttttttatt'

for seq in [seq1, seq2]:
    log_likelihood, path = model.viterbi([*seq])
    print("The most likely underlying series of exons/introns states is {} at {:.250}%."
        .format([s[1].name for s in path[1:]], np.exp(log_likelihood)*100))

The most likely underlying series of exons/introns states is ['exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns'] at 2.0097848153611882792928659323301341577734262987639678293950128151745815330286859534680843353271484375e-21%.
The most likely underlying series of exons/introns states is ['introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'introns', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exons', 'exon

> Remark: the likelihood is extremely small because of the strict conditions of the probability